In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut

path = "data/ARIA/7.ActivityAndBilling/"

show = 0
sch = ut.leer_csv("data/ARIA/9.Schedule/_ScheduledActivity.csv", show).drop(columns=["ActivityInstanceRevCount","CreationDate",
      "CreatedByResourceSer","ActivityNote","ScheduledActivityRevCount","CreatedByUserName","UID","RecurrenceRuleSer",
       "WaitListedFlag","WorkFlowActiveFlag"])

acd = ut.leer_csv(path+"_ActCaptDiagnosis.csv", show).drop(columns=["ActivityCaptureRevCount","ActCaptDiagnosisRevCount",
                             "ObjectStatus","LastKnownDiagnosisId","LastKnownDiagnosisCode"])

cap = ut.leer_csv(path+"_ActivityCapture.csv", show).drop(columns=["ActivityCaptureRevCount","ActivityInstanceRevCount",
                                              "LastKnownCourseId","InPatientFlag","AttendingOncologistSer","ResourceSer"])

ins = ut.leer_csv(path+"_ActivityInstance.csv", show).drop(columns=["ActivityInstanceRevCount","ActivityRevCount",
       "PredecessorSer","TemplateCycleRevCount","ActivityInstanceId","WorkFlowOverrideByResourceSer","WorkFlowOverrideDate",
      "TemplateCycleSer","LastNoteUpdatedByResourceSer","LastStatusUpdatedByResourceSer","DepartmentSer","LastNoteUpdatedDate",
      "LastStatusUpdatedDate", "AppointmentDependentFlag"])

a = ut.leer_csv(path+"_Activity.csv", show).drop(columns=["ActivityRevCount","UpdCPResourceGroupSer","DerivedFromActivitySer",
      "UpdCPEscalationGroupSer","EscalationGroupSer","ActivityType","ForeGroundColor","DICOMCodeValueSer","TreatmentCycleSer"])

cat = ut.leer_csv(path+"_ActivityCategory.csv", show).drop(columns=["DepartmentSer","DerivedFromActCategorySer"])

att = ut.leer_csv(path+"Attendee.csv", show).drop(columns=["AttendeeRevCount","ActivityInstanceRevCount"])
res = ut.leer_csv("data/ARIA/6.Resource/Resource.csv", show).drop(columns=["DICOMCodeValueSer"])
mach = ut.leer_csv("data/ARIA/12.Machine/Machine.csv", show)
tr = ut.leer_csv("data/ARIA/5.CourseOfTreatment/TreatmentRecord.csv", show).drop(columns=["TreatmentRecordUID","PlanUID",
          "ActualMachineAuthorization","FileName","CharacterSet"])
diag = ut.leer_csv("data/ARIA_cleaned/Diagnosis_pri.csv", show)

D:\miniconda3\envs\tfgpu\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
# ut.info_tabla(tr)

In [2]:
sch["ScheduledStartTime"] = pd.to_datetime(sch["ScheduledStartTime"])
sch["ScheduledEndTime"] = pd.to_datetime(sch["ScheduledEndTime"])
sch["ActualStartDate"] = pd.to_datetime(sch["ActualStartDate"])
sch["ActualEndDate"] = pd.to_datetime(sch["ActualEndDate"])

In [3]:
print(ins.shape)
print(cap.shape)
print(sch.shape)
print(acd.shape)
print()
print(a.shape)
print(cat.shape)

(302025, 10)
(181361, 4)
(183624, 10)
(121107, 3)

(261, 15)
(76, 5)


In [4]:
final = sch \
.merge(ins, on="ActivityInstanceSer", how="left") \
.merge(cap, on="ActivityInstanceSer", how="left") \
.merge(acd, on="ActivityCaptureSer", how="left") \
.merge(a,   on="ActivitySer", how="left") \
.merge(cat.drop(columns="ObjectStatus"), on="ActivityCategorySer", how="left")

final = final.drop(columns=["ActivityInstanceSer","AppointmentInstanceFlag","ActivitySer","ActivityCaptureSer",
                       "ActCaptDiagnosisSer","ActivityCategorySer","NoAutoPostFlag","AutoGenerateFlag"])

In [ ]:
# ut.info_tabla(final)

In [5]:
final.to_csv("data/ARIA_cleaned/SchActivity.csv", index=False)

# Citas tratamiento

In [ ]:
citas = final[final.ActualEndDate.notnull() & final.ActualStartDate.notnull() & (final.ActivityCategoryCode=="Treatment")]\
.drop(columns=["ObjectStatus_x","PriorPostDueDurUnits","ObjectStatus_y","AutoAssignOncologistFlag","ReviewFlag",
              "ExclusiveFlag","ObjectStatus","AutoGenerateFlag","NoAutoPostFlag","ActivityCategoryCode","SchedulableFlag",
              "NoEditFlag_y","ScheduledActivityCode","AnchorActivityFlag"])

print(citas.shape)
citas.isna().sum()
# citas.nunique()
# citas.DefaultDuration.value_counts()

citas[citas.CourseSer.isna()]